In [1]:
from torch.utils.data import DataLoader
import torch

import minari
import numpy as np
from typing import Dict

import os
import sys
sys.path.append("..")
from algorithms.utils.wrapper_gym import get_env
from algorithms.utils.dataset import qlearning_dataset, ReplayBuffer


In [2]:
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [3]:
os.environ["MINARI_DATASETS_PATH"] = "/home/luanagbmartins/Documents/CEIA/offline_to_online/CORL/datasets"

In [4]:
DATSET_NAME = "playground/Go1JoystickRoughTerrain-expert-v0"
ENV_NAME = "Go1JoystickRoughTerrain"

DEVICE = "cuda"

In [5]:
dataset = minari.load_dataset("playground/Go1JoystickRoughTerrain-expert-v0")

In [6]:
qdataset = qlearning_dataset(dataset)

# Create the replay buffer and dataloader
replay_buffer = ReplayBuffer(qdataset)
dataloader = DataLoader(replay_buffer, batch_size=128, shuffle=True)

# Now you can iterate over the dataloader
batch = next(iter(dataloader))

In [7]:
state, action, reward, next_state, done = batch

In [8]:
state.shape

torch.Size([128, 48])

In [9]:
env = get_env(ENV_NAME, DEVICE)
action_dim = env.action_space.shape[0]

episode_rewards = []
for _ in range(10):
    state, _ = env.reset()
    done = False
    episode_reward = 0.0
    while not done:
        action = np.random.randn(action_dim)
        state, reward, done, _, _ = env.step(action)
        episode_reward += reward
    episode_rewards.append(episode_reward)

obs_shape: {'privileged_state': (123,), 'state': (48,)}
Asymmetric observation space
JITing reset and step
Done JITing reset and step


In [10]:
np.asarray(episode_rewards).mean(), np.asarray(episode_rewards).std()

(np.float64(0.0), np.float64(0.0))